<a href="https://colab.research.google.com/github/tayfununal/2-Hidden-Layer-Neural-Networks/blob/master/one_hidden_layer_Exp_NoReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playground-data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plygdata as pg
import json
import requests

In [ ]:
Datas = json.loads(requests.get("https://raw.githubusercontent.com/tayfununal/2-Hidden-Layer-Neural-Networks/master/produced_XOR_Datas.json").text)

In [ ]:
df = pd.DataFrame(Datas["1"])
df

In [ ]:
X = np.array(df[[0,1]]).T
Y = np.array(df[[2]]).T

In [ ]:
print("Shape of X_train:", X.shape,
      "\nShape of y_train:", Y.shape)

In [ ]:
def initialization_parameters(x, y, num_node, random_seeds_one, random_seeds_two):
    np.random.seed(random_seeds_one)
    W1 = np.random.randn(num_node * x.shape[0]).reshape(num_node, x.shape[0]) # *0.001
    b1 = np.zeros((num_node, 1))

    np.random.seed(random_seeds_two)
    W2 = np.random.randn(y.shape[0], num_node) #*0.001
    b2 = np.zeros((y.shape[0], 1))

    assert W1.shape == (num_node, x.shape[0])
    assert b1.shape == (num_node, 1)

    assert W2.shape == (y.shape[0], num_node)
    assert b2.shape == (y.shape[0], 1)

    parameters = {'W1': W1,
                  'b1': b1,
                  'W2': W2,
                  'b2': b2}
    return parameters


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def relu(x):
    return np.maximum(0, x)


def reluDerivative(x):
    x[x <= 0] = 0
    x[x > 0] = 1
    return x


def forward_prop(x, parameters):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']

    Z1 = np.dot(W1, x) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)

    assert (A2.shape == (1, x.shape[1]))
    cache = {
        'Z1': Z1,
        'A1': A1,
        'Z2': Z2,
        'A2': A2
    }
    return A2, cache


def backward_prop(x, y, parameters, cache, learning_rate=0.1):
    m = y.shape[1]

    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']

    A1 = cache['A1']
    A2 = cache['A2']

    # Calculations of backward propagation: dW1, db1, dW2, db2
    dZ2 = A2 - y
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = np.multiply(np.dot(W2.T, dZ2), reluDerivative(A1))
    dW1 = (1 / m) * np.dot(dZ1, x.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)

    # Updating parameters
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2

    parameters = {'W1': W1,
                  'b1': b1,
                  'W2': W2,
                  'b2': b2}
    return parameters


def cross_entropy_cost(y, A2, parameters):
    m = y.shape[1]

    W1 = parameters["W1"]
    W2 = parameters["W2"]

    cross_entropy = np.multiply(np.log(A2 + 1e-15), y) + np.multiply((1 - y), np.log(1 - A2 + 1e-15))
    cost = (- 1.0 / m) * np.sum(cross_entropy)

    # Squeezing to avoid unnecessary dimensions
    cost = np.squeeze(cost)
    return cost


def accuracy(X, Y, parameters):
    A2, cache = forward_prop(X, parameters)
    predictions = np.round(A2)
    accuracy = float((np.dot(Y, predictions.T) + np.dot(1 - Y, 1 - predictions.T)) / float(Y.size) * 100)
    return accuracy

In [ ]:
def nn_model(x, y, parameters, number_of_iter=1000):
    A2, cache = forward_prop(x, parameters)
    cost = cross_entropy_cost(y, A2, parameters)

    for i in range(1, number_of_iter):
        parameters = backward_prop(x, y, parameters, cache, learning_rate=0.6)
        A2, cache = forward_prop(x, parameters)
        cost = cross_entropy_cost(y, A2, parameters)

        # cost_value[i+1] = cost
        """
        if (i+1) % 1000 == 0:
         cost_value[i+1] = cost
         print("%i."%(i+1),cost)
         """
    return cost, parameters

In [ ]:
np.random.seed(65468)
random_seeds = np.zeros((100,2))
for i in range(0,100):
  for j in range(0,2):
    random_seeds[i,j] = np.random.randint(1,100001)

In [ ]:
pg.dataset.random.seed(2)
test_data = pg.dataset.DataGenerator.classify_xor(1000,noise=0.1)
test_data = pd.DataFrame(test_data , columns=["x1","x2","y"])
test_data_X = np.array(test_data[['x1','x2']]).T
test_data_Y = np.array(test_data[['y']]).T
test_data_Y[test_data_Y==-1] = 0

In [ ]:
from collections import Counter
average_aHundred_cost = dict()
test_cost = dict()
average_aHundred_test_cost = dict()
test_data_result = dict()

for n in range(1,101): #101 #Toplam 100 tane veri kümesi olduğundan 100 kere çalıçmasını sağlamak için 101 e kadar saydırmalıyız.
  df = pd.DataFrame(Datas["{}".format(n)])
  X = np.array(df[[0,1]]).T
  Y = np.array(df[[2]]).T

  for i in range(0,100): #100 initial point
    average_twenty_cost = dict()
    for number_of_neuron in [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]:
      parameters = initialization_parameters(X, Y, number_of_neuron, int(random_seeds[i,0]), int(random_seeds[i,1]))
      cost, parameters = nn_model(X , Y, parameters, number_of_iter = 1000)
      average_twenty_cost['{}'.format(number_of_neuron)] = cost * 1 / 100 #100 tane farklı başlangıç noktası ile modeli eğittiğimiz için 100'e bölüyoruz.

      A2, cache = forward_prop(test_data_X, parameters)
      test_cost['{}'.format(number_of_neuron)] = cross_entropy_cost(test_data_Y, A2, parameters) * 1 / 100
    average_aHundred_test_cost = Counter(average_aHundred_test_cost) + Counter(test_cost)
    average_aHundred_cost = Counter(average_aHundred_cost) + Counter(average_twenty_cost)
  if (n % 5) == 0:
    print(n," - controlTEST",average_aHundred_test_cost,"controlCOST",average_aHundred_cost)

In [ ]:
#print("ty",test_cost)
print("test100", dict(average_aHundred_test_cost))
#print("average20",average_twenty_cost)
print("cost100",dict(average_aHundred_cost))

In [ ]:
"""

x = [i for i in range(1,21)]
y= np.array(list(average_aHundred_cost.values()))
z = np.array(list(test_cost.values()))
plt.plot(x, y, color="red")
plt.plot(x,z, color="blue")
plt.show()

"""


"""

test100 {'1': 56.07265429945398, '5': 4.284047276946905, '10': 3.636714606428911, '15': 3.6698393130692955, '20': 3.759570419142598, '25': 3.934039975779941, '30': 4.144499426368325, '35': 4.428522087632986, '40': 4.742464434685483, '45': 5.153614120076064, '50': 5.558837648962923, '55': 5.9800149737490855, '60': 6.38212777646587, '65': 6.760059681686107, '70': 7.208347398526732, '75': 7.63743241415394, '80': 7.953036280424238, '85': 8.391668624621495, '90': 8.87750885773311, '95': 9.401192662393102, '100': 9.781579503271404}
cost100 {'1': 55.54437869906683, '5': 3.140898047740914, '10': 2.3695972533199905, '15': 2.3153408685157664, '20': 2.274080535742187, '25': 2.2495886017608675, '30': 2.2383628403263724, '35': 2.245431895774773, '40': 2.2698020126412266, '45': 2.3194943853538628, '50': 2.3773174388052434, '55': 2.4424143734230856, '60': 2.51119149402324, '65': 2.569740921100265, '70': 2.6562294970087685, '75': 2.762468858008673, '80': 2.7961181666818815, '85': 2.8940388503625614, '90': 3.0024958271468543, '95': 3.117988699809893, '100': 3.201184186388983}

"""